In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("Trial_dataset.csv")
df

,Statement,Deceptive
0,No sir I did not. I absolutely did not. No sir...,1
1,"... and she approached me, and at that time th...",1
2,"No sir I was not, not at all.",1
3,"He had told me that he had had a dream that, a...",1
4,"And he told me that, ammm � he was trying to f...",1
...,...,...
115,"I was born in Durango, Colorado. My mother was...",0
116,"Um whenever I was little, um my oldest nephew ...",0
117,"We were married for two years, and um, we divo...",0
118,"Yes um, Reginald and I would have like long ta...",0


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words("english"))

def remove_stopWords(s):
 
    s = ' '.join(word for word in s.split() if word not in stop_words)
    return s

df.loc[:,"Statement"] = df.Statement.apply(lambda x: remove_stopWords(x))

In [ ]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
def normalize(text):
    text = text.replace('\n', '')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.replace('+', ' ')
    return text

X = df['Statement'].apply(normalize)

In [ ]:
df

,Statement,Deceptive
0,No sir I not. I absolutely not. No sir I not. ...,1
1,"... approached me, time tripped rug kind bumpe...",1
2,"No sir I not, all.",1
3,"He told dream that, ammm ...he forest killed L...",1
4,"And told that, ammm � trying figure stuff out,...",1
...,...,...
115,"I born Durango, Colorado. My mother single mot...",0
116,"Um whenever I little, um oldest nephew um two ...",0
117,"We married two years, um, divorced 10 months o...",0
118,"Yes um, Reginald I would like long talks one l...",0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Statement'],df['Deceptive'],test_size=0.1)

In [ ]:
def stem_text(text):
    import nltk
    from nltk.tokenize import word_tokenize
    from nltk.stem.isri import ISRIStemmer

    stemmer = ISRIStemmer()
    tokens = word_tokenize(text)
    
    return ' '.join([stemmer.stem(t) for t in tokens])

In [ ]:
with open("glove.6B.100d.txt", "rb") as lines:
    gloves = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}
gloves

{b'the': array(<map object at 0x7ff5aa393310>, dtype=object),
 b',': array(<map object at 0x7ff5aa393390>, dtype=object),
 b'.': array(<map object at 0x7ff5aa393410>, dtype=object),
 b'of': array(<map object at 0x7ff5aa393490>, dtype=object),
 b'to': array(<map object at 0x7ff5aa393510>, dtype=object),
 b'and': array(<map object at 0x7ff5aa393590>, dtype=object),
 b'in': array(<map object at 0x7ff5aa393610>, dtype=object),
 b'a': array(<map object at 0x7ff5aa393690>, dtype=object),
 b'"': array(<map object at 0x7ff5aa393710>, dtype=object),
 b"'s": array(<map object at 0x7ff5aa393790>, dtype=object),
 b'for': array(<map object at 0x7ff5aa393810>, dtype=object),
 b'-': array(<map object at 0x7ff5aa393890>, dtype=object),
 b'that': array(<map object at 0x7ff5aa393910>, dtype=object),
 b'on': array(<map object at 0x7ff5aa393990>, dtype=object),
 b'is': array(<map object at 0x7ff5aa389f90>, dtype=object),
 b'was': array(<map object at 0x7ff5aa389d90>, dtype=object),
 b'said': array(<map ob

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict


## define two classes to write functions of fit & transform (to be able to include w2v in pipeline)
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.items())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.items())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

scv_clf = SVC()
DT_clf = DecisionTreeClassifier(criterion='entropy', max_depth=4)
Ada_clf = AdaBoostClassifier(n_estimators=100)
KNN_clf = KNeighborsClassifier( n_neighbors=5)
RF_clf = RandomForestClassifier()

In [ ]:
from sklearn.metrics import classification_report  
def print_measures(y_test, y_pred):
    print(classification_report(y_test, y_pred))

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from sklearn.pipeline import Pipeline

#Define Pipeline
gloves_pipeline =Pipeline([('gloves', TfidfEmbeddingVectorizer(gloves)),('SCV_clf', scv_clf)])

#train
gloves_pipeline.fit(X_train.apply(stem_text),y_train)

#test
print('For gloves__pipeline')
print_measures(y_test, gloves_pipeline.predict(X_test.apply(stem_text)))


For gloves__pipeline
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.33      1.00      0.50         4

    accuracy                           0.33        12
   macro avg       0.17      0.50      0.25        12
weighted avg       0.11      0.33      0.17        12



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
from sklearn.pipeline import Pipeline

#Define Pipeline
gloves_pipeline =Pipeline([('gloves', TfidfEmbeddingVectorizer(gloves)),('DT_clf', DT_clf)])

#train
gloves_pipeline.fit(X_train.apply(stem_text),y_train)

#test
print('For gloves__pipeline')
print_measures(y_test, gloves_pipeline.predict(X_test.apply(stem_text)))


For gloves__pipeline
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.33      1.00      0.50         4

    accuracy                           0.33        12
   macro avg       0.17      0.50      0.25        12
weighted avg       0.11      0.33      0.17        12



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
from sklearn.pipeline import Pipeline

#Define Pipeline
gloves_pipeline =Pipeline([('gloves', TfidfEmbeddingVectorizer(gloves)),('RF_clf', RF_clf)])

#train
gloves_pipeline.fit(X_train.apply(stem_text),y_train)

#test
print('For gloves__pipeline')
print_measures(y_test, gloves_pipeline.predict(X_test.apply(stem_text)))


For gloves__pipeline
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.33      1.00      0.50         4

    accuracy                           0.33        12
   macro avg       0.17      0.50      0.25        12
weighted avg       0.11      0.33      0.17        12



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
from sklearn.pipeline import Pipeline

#Define Pipeline
gloves_pipeline =Pipeline([('gloves', MeanEmbeddingVectorizer(gloves)),('KNN_clf', KNN_clf)])

#train
gloves_pipeline.fit(X_train.apply(stem_text),y_train)

#test
print('For gloves__pipeline')
print_measures(y_test, gloves_pipeline.predict(X_test.apply(stem_text)))


For gloves__pipeline
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.33      1.00      0.50         4

    accuracy                           0.33        12
   macro avg       0.17      0.50      0.25        12
weighted avg       0.11      0.33      0.17        12



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
from sklearn.pipeline import Pipeline

#Define Pipeline
gloves_pipeline =Pipeline([('gloves', MeanEmbeddingVectorizer(gloves)),('Ada_clf', Ada_clf)])

#train
gloves_pipeline.fit(X_train.apply(stem_text),y_train)

#test
print('For gloves__pipeline')
print_measures(y_test, gloves_pipeline.predict(X_test.apply(stem_text)))


For gloves__pipeline
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.33      1.00      0.50         4

    accuracy                           0.33        12
   macro avg       0.17      0.50      0.25        12
weighted avg       0.11      0.33      0.17        12



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

